In [1]:
import numpy as np
import pandas as pd
import SolveLinSys
import matplotlib.pyplot as plt
from numba import njit

In [2]:
@njit
def compute_derivatives(data, dim, order, dlt):
    res = np.zeros_like(data)
    if order == 1:                    # first order derivatives
        if dim == 0:                  # to first dimension
            res[1:-1,:] = (1 / (2 * dlt)) * (data[2:,:] - data[:-2,:])
            res[-1,:] = (1 / dlt) * (data[-1,:] - data[-2,:])
            res[0,:] = (1 / dlt) * (data[1,:] - data[0,:])
        elif dim == 1:                # to second dimension
            res[:,1:-1] = (1 / (2 * dlt)) * (data[:,2:] - data[:,:-2])
            res[:,-1] = (1 / dlt) * (data[:,-1] - data[:,-2])
            res[:,0] = (1 / dlt) * (data[:,1] - data[:,0])        
    elif order == 2:
        if dim == 0:                  # to first dimension
            res[1:-1,:] = (1 / dlt ** 2) * (data[2:,:] + data[:-2,:] - 2 * data[1:-1,:])
            res[-1,:] = (1 / dlt ** 2) * (data[-1,:] + data[-3,:] - 2 * data[-2,:])
            res[0,:] = (1 / dlt ** 2) * (data[2,:] + data[0,:] - 2 * data[1,:])
        elif dim == 1:                # to second dimension
            res[:,1:-1] = (1 / dlt ** 2) * (data[:,2:] + data[:,:-2] - 2 * data[:,1:-1])
            res[:,-1] = (1 / dlt ** 2) * (data[:,-1] + data[:,-3] - 2 * data[:,-2])
            res[:,0] = (1 / dlt ** 2) * (data[:,2] + data[:,0] - 2 * data[:,1])    
    return res

def false_transient_one_iteration(stateSpace, A, B1, B2, C1, C2, D, v0, ε=.3, tol=-10):
    A = A.reshape(-1, 1, order='F')
    B = np.hstack([B1.reshape(-1, 1, order='F'), B2.reshape(-1, 1, order='F')])
    C = np.hstack([C1.reshape(-1, 1, order='F'), C2.reshape(-1, 1, order='F')])
    D = D.reshape(-1, 1, order='F')
    v0 = v0.reshape(-1, 1, order='F')
    out = SolveLinSys.solveFT(stateSpace, A, B, C, D, v0, ε, tol)
    return out

PDE:
\begin{align*}
0 = \max_{\tilde{e}}  &  -\delta \phi(y_1, y_2) + \delta \eta \log \mathcal{E} \\
& + \frac{\partial \phi}{\partial y_1} y_2 + \frac{\partial \phi}{\partial y_2} (-\lambda y_2 + \lambda \theta \mathcal{E})\\
&+ (\eta-1)\left[\gamma_1 + \gamma_2 y_1 + \gamma_2^+\boldsymbol{1}_{y_1\geq \bar{y}_1}(y_1-\bar{y}_1)\right]y_2\\
\end{align*}

FOC of $\mathcal{E}$ gives:
$$
\mathcal{E} = -\frac{\delta \eta}{\frac{\partial \phi}{\partial y_2} \lambda \theta}
$$

In [10]:
def false_transient(y1_grid, y2_grid, model_paras=(), v0=None, ϵ=.5, tol=1e-8, max_iter=10_000, print_all=True):
    η, δ, θ, γ_1, γ_2, γ_2p, y_bar = model_paras
    dy1 = y1_grid[1] - y1_grid[0]
    dy2 = y2_grid[1] - y2_grid[0]
    (y1_mat, y2_mat) = np.meshgrid(y1_grid, y2_grid, indexing = 'ij')
    stateSpace = np.hstack([y1_mat.reshape(-1, 1, order='F'), y2_mat.reshape(-1, 1, order='F')])    

    if v0 is None:
        v0 = -δ*η*(y2_mat+y2_mat**2)

    d_Λ = γ_1 + γ_2*y1_mat + γ_2p*(y1_mat>y_bar)*(y1_mat-y_bar)

    count = 0
    error = 1.

    while error > tol and count < max_iter:
        v_old = v0.copy()

        v0_dy1 = compute_derivatives(v0, 0, 1, dy1)
        v0_dy2 = compute_derivatives(v0, 1, 1, dy2)

        e_tilde = -δ*η/(v0_dy2*λ*θ)
        e_tilde[e_tilde<=0] = 1e-16

        A = np.ones_like(y1_mat) * (-δ)
        B1 = y2_mat
#         B2 = λ*(-y2_mat+θ*e_tilde)
        B2 = λ*(-y2_mat)
        C1 = np.zeros_like(y1_mat)
        C2 = np.zeros_like(y2_mat)
#         D = δ*η*np.log(e_tilde) + (η-1)*d_Λ*y2_mat
        D = δ*η*np.log(e_tilde) + (η-1)*d_Λ*y2_mat - δ*η

        res = false_transient_one_iteration(stateSpace, A, B1, B2, C1, C2, D, v0, ϵ) # CG tol = 1e-10
        v0 = res[2].reshape(v0.shape, order = "F")

        rhs_error = A*v0 + B1*v0_dy1 + B2*v0_dy2 + D
        rhs_error = np.max(abs(rhs_error))
        lhs_error = np.max(abs((v0 - v_old)/ϵ))
        error = lhs_error
        count += 1
        if print_all:
            print("Iteration %s: LHS Error: %s; RHS Error %s" % (count, lhs_error, rhs_error))

    print("Converged. Total iteration %s: LHS Error: %s; RHS Error %s" % (count, lhs_error, rhs_error))

    res = {'v0': v0,
           'e_tilde': e_tilde}
    return res

In [12]:
λ = - np.log(0.5)/5

η = .032
δ = .01

θ = pd.read_csv('../data/model144.csv', header=None).to_numpy()[:, 0]/1000.
θ = np.mean(θ)

y_bar = 2.
γ_1 = 1.7675/10000
γ_2 = .0022*2
# γ_2p = .0197*2*.5 # weighted model of low and high
γ_2p = 0.

n_y1 = 50
y1_min = 0.
y1_max = 4.
y1_grid = np.linspace(y1_min, y1_max, n_y1)

n_y2 = 50
y2_min = 0.
y2_max = 4.
y2_grid = np.linspace(y2_min, y2_max, n_y2)

model_paras = (η, δ, θ, γ_1, γ_2, γ_2p, y_bar)
model_res = false_transient(y1_grid, y2_grid, model_paras,
                            v0=None, ϵ=1., tol=1e-8, max_iter=5_000, print_all=True)

Iteration 1: LHS Error: 0.10318615744062216; RHS Error 0.06452951854299513
Iteration 2: LHS Error: 0.1266652295373293; RHS Error 0.10276543970295654
Iteration 3: LHS Error: 0.13865154012875855; RHS Error 0.12553859997428565
Iteration 4: LHS Error: 0.14281240783358262; RHS Error 0.13738127149525606
Iteration 5: LHS Error: 0.1415792758407759; RHS Error 0.14150701941694824
Iteration 6: LHS Error: 0.13673417097469698; RHS Error 0.1402944839249683
Iteration 7: LHS Error: 0.12958275104040196; RHS Error 0.13550255204035572
Iteration 8: LHS Error: 0.12107380806634238; RHS Error 0.1284232761624286
Iteration 9: LHS Error: 0.11188824595489821; RHS Error 0.1199966165255657
Iteration 10: LHS Error: 0.1025064762120731; RHS Error 0.11089755731300871
Iteration 11: LHS Error: 0.09325939512060422; RHS Error 0.10160251921558383
Iteration 12: LHS Error: 0.08436737196089594; RHS Error 0.09243977613000808
Iteration 13: LHS Error: 0.0759695493946102; RHS Error 0.08362801371614939
Iteration 14: LHS Error: 0.0

Iteration 109: LHS Error: 0.00043297871752789874; RHS Error 0.004054229008431347
Iteration 110: LHS Error: 0.00042754030605829385; RHS Error 0.004058679359003281
Iteration 111: LHS Error: 0.0004220962511980453; RHS Error 0.004063083194033013
Iteration 112: LHS Error: 0.0004167771550451782; RHS Error 0.004067434798541066
Iteration 113: LHS Error: 0.0004115249576317481; RHS Error 0.004071725425285874
Iteration 114: LHS Error: 0.0004063660048756468; RHS Error 0.00407594896062093
Iteration 115: LHS Error: 0.0004013015362374084; RHS Error 0.004080101758961974
Iteration 116: LHS Error: 0.0003962871130780421; RHS Error 0.004084185661592673
Iteration 117: LHS Error: 0.0003913652637903553; RHS Error 0.004088208099316618
Iteration 118: LHS Error: 0.0003864630132084379; RHS Error 0.004092178063672196
Iteration 119: LHS Error: 0.00038164585712863186; RHS Error 0.00409610344823352
Iteration 120: LHS Error: 0.00037686711845719145; RHS Error 0.004099989695549059
Iteration 121: LHS Error: 0.0003721123

Iteration 211: LHS Error: 0.00012224206010369443; RHS Error 0.004307477135018738
Iteration 212: LHS Error: 0.00012073495430930947; RHS Error 0.004308696638242349
Iteration 213: LHS Error: 0.000119212963374693; RHS Error 0.0043099265461265315
Iteration 214: LHS Error: 0.00011769734001676468; RHS Error 0.004311154086558372
Iteration 215: LHS Error: 0.00011626122293773022; RHS Error 0.004312365326768193
Iteration 216: LHS Error: 0.00011483104723097881; RHS Error 0.004313543263750075
Iteration 217: LHS Error: 0.00011347587798304914; RHS Error 0.0043146772756163685
Iteration 218: LHS Error: 0.0001121697693822199; RHS Error 0.00431576496902647
Iteration 219: LHS Error: 0.00011092656921585231; RHS Error 0.004316812277113802
Iteration 220: LHS Error: 0.00010966651099364455; RHS Error 0.004317829151346113
Iteration 221: LHS Error: 0.00010843729206078123; RHS Error 0.004318826668367585
Iteration 222: LHS Error: 0.00010719177847962058; RHS Error 0.004319815700342711
Iteration 223: LHS Error: 0.00

Iteration 317: LHS Error: 2.0235748500629924e-05; RHS Error 0.0044252849855578175
Iteration 318: LHS Error: 4.39695597223011e-05; RHS Error 0.004419788093747726
Iteration 319: LHS Error: 5.870231804339099e-05; RHS Error 0.004386689119180969
Iteration 320: LHS Error: 7.919204133477464e-05; RHS Error 0.004370150763728144
Iteration 321: LHS Error: 8.866828933262738e-05; RHS Error 0.004356771423486101
Iteration 322: LHS Error: 7.230464936602843e-05; RHS Error 0.004352060203633627
Iteration 323: LHS Error: 5.3290916072423755e-05; RHS Error 0.004358442989355452
Iteration 324: LHS Error: 1.8921175581665395e-05; RHS Error 0.00436996825406819
Iteration 325: LHS Error: 1.2729256317456361e-05; RHS Error 0.004402336280616426
Iteration 326: LHS Error: 1.779683739588922e-05; RHS Error 0.004411359616895635
Iteration 327: LHS Error: 2.0507624975740768e-05; RHS Error 0.004412421393673871
Iteration 328: LHS Error: 2.1411928403658465e-05; RHS Error 0.004419341708845725
Iteration 329: LHS Error: 2.1569001

Iteration 422: LHS Error: 1.2612122323729658e-05; RHS Error 0.004408330331478631
Iteration 423: LHS Error: 1.8095555741615144e-05; RHS Error 0.0044100961029551675
Iteration 424: LHS Error: 1.9869628865576416e-05; RHS Error 0.0044182720901210365
Iteration 425: LHS Error: 1.829746280535005e-05; RHS Error 0.004424518514395723
Iteration 426: LHS Error: 1.2732735181542054e-05; RHS Error 0.004421773538072035
Iteration 427: LHS Error: 5.421580873798959e-06; RHS Error 0.004418064704049207
Iteration 428: LHS Error: 7.13611897529276e-06; RHS Error 0.00441206065719453
Iteration 429: LHS Error: 9.317903599259658e-06; RHS Error 0.004407786408069644
Iteration 430: LHS Error: 8.436508220022176e-06; RHS Error 0.004403731382202348
Iteration 431: LHS Error: 1.1395310943074222e-05; RHS Error 0.004404765135732895
Iteration 432: LHS Error: 2.1199446225139695e-05; RHS Error 0.004405389125292758
Iteration 433: LHS Error: 3.5142787803366105e-05; RHS Error 0.004410739288831775
Iteration 434: LHS Error: 5.50228

Iteration 525: LHS Error: 1.991505709231234e-06; RHS Error 0.00440952476099242
Iteration 526: LHS Error: 1.7138006077610513e-06; RHS Error 0.004409009896981356
Iteration 527: LHS Error: 1.4250360971246323e-06; RHS Error 0.004408094727018849
Iteration 528: LHS Error: 1.2858143101368569e-06; RHS Error 0.00440754439378635
Iteration 529: LHS Error: 8.205601210731217e-07; RHS Error 0.004407763730083052
Iteration 530: LHS Error: 7.242158078235583e-07; RHS Error 0.004408852467937804
Iteration 531: LHS Error: 9.455027903393365e-07; RHS Error 0.004408959390710468
Iteration 532: LHS Error: 1.1299969087108508e-06; RHS Error 0.004408862045802625
Iteration 533: LHS Error: 1.3471012552668249e-06; RHS Error 0.0044088546677527735
Iteration 534: LHS Error: 1.5427887492536208e-06; RHS Error 0.004408841333186891
Iteration 535: LHS Error: 2.0143865153143814e-06; RHS Error 0.004408687623720112
Iteration 536: LHS Error: 2.336557394011507e-06; RHS Error 0.0044086151909526194
Iteration 537: LHS Error: 2.83557

Iteration 717: LHS Error: 4.591645139107037e-08; RHS Error 0.004408886577114857
Iteration 718: LHS Error: 4.525230237750755e-08; RHS Error 0.004408886692635325
Iteration 719: LHS Error: 4.4597097653698015e-08; RHS Error 0.004408886815841978
Iteration 720: LHS Error: 4.3950182708474284e-08; RHS Error 0.004408886930157466
Iteration 721: LHS Error: 4.3312560295272196e-08; RHS Error 0.004408887052770191
Iteration 722: LHS Error: 4.268307229188251e-08; RHS Error 0.00440888716630744
Iteration 723: LHS Error: 4.2062669881881565e-08; RHS Error 0.0044088872880028995
Iteration 724: LHS Error: 4.145054855603236e-08; RHS Error 0.0044088874009478585
Iteration 725: LHS Error: 4.084699668782665e-08; RHS Error 0.004408887521817603
Iteration 726: LHS Error: 4.0587573364725404e-08; RHS Error 0.004408887634089531
Iteration 727: LHS Error: 3.9683992730799034e-08; RHS Error 0.004408887743254417
Iteration 728: LHS Error: 3.911287704133626e-08; RHS Error 0.004408887854098181
Iteration 729: LHS Error: 3.85509

In [ ]:
def simulation(y_grid, e_grid, θ=1.86/1000., y_start=.5394, T=100):
    Et = np.zeros(T+1)
    yt = np.zeros(T+1)
    for i in range(T+1):
        Et[i] = np.interp(y_start, y_grid, e_grid)
        yt[i] = y_start
        y_start = y_start + Et[i]*θ
    return Et, yt

In [ ]:
et_neutral, _ = simulation(y_grid, e_tilde_list[0])
et_averse, _ = simulation(y_grid, e_tilde_list[1])
plt.plot(et_neutral)
plt.plot(et_averse)

**ODE for z**:
$$
0 = \min_h -\delta \eta \log z_2 + \frac{\xi_{1,m}}{2}h^\prime h + \frac{d\psi(z_2)}{d z_2}\left[-\rho(z_2-\mu_2) + \sqrt{z_2}\sigma_2 \cdot h\right] + \frac{1}{2}\frac{d^2 \psi(z_2)}{(d z_2)^2}z_2|\sigma_2|^2
$$

The FOC of $h$ gives:
$$
h^* = -\frac{\frac{d\psi(z_2)}{d z_2}\sqrt{z_2}\sigma_2}{\xi_{1,m}}
$$

In [ ]:
# Model parameters
η = .032
δ = .01

ρ = 0.9
μ_2 = 1.
σ_2 = 0.013145579 / np.sqrt(1.86/1000)

ξ_1m = 1000.

n_z = 100
z_min = np.max([μ_2 - 3*np.sqrt(σ_2**2*μ_2/(2*ρ)), 1e-9])
z_max = μ_2 + 3*np.sqrt(σ_2**2*μ_2/(2*ρ))
z_grid = np.linspace(z_min, z_max, n_z)

model_paras = (η, δ, ρ, μ_2, σ_2, ξ_1m) 
ψ = ode_z(z_grid, model_paras, v0=None, ϵ=.5, tol=1e-8, max_iter=2_000)

In [ ]:
plt.plot(z_grid, ψ)